In [15]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

import seaborn as sns
import matplotlib as mpl
import matplotlib . pyplot as plt

import pprint

# LoL Predictor

## Dataset

Le set de donnée que nous avons choisi est [League Of Legends High Elo Ranked Games](https://www.kaggle.com/gyejr95/league-of-legends-challenger-ranked-games2020). 

Ce set de donnée représente les données de 26 000 parties des meilleurs joueurs de League Of Legends, 65 000 parties des 2nds meilleurs joueurs et 100 000 parties des 3ème meilleurs.

League Of Legends est un jeu de 5vs5 consistant à détruire la base de l'autre. En éliminant les joueurs adverses ou minions/objectifs, les joueurs remportent de l'expérience leur permettant d'augmenter de niveau et aussi de l'argent.

En accomplissant des objectifs, une équipe a plus de chance de gagner la partie. 

Nous allons essayer de prédire si une équipe va gagner en fonction de son état dans la partie.

Nous allons uniquement nous baser sur les 

## Data

Une ligne de données est composé en deux parties : 

* Equipe bleu
* Equipe rouge

Ces deux ensembles sont composés des mêmes colonnes, ici nous allons étudier que l'équipe bleu.


Les différentes colonnes sont les suivantes : 

In [19]:
# Import challenger and grandmaster data
challenger_data = pd.read_csv('data/challenger.csv')
grandmaster_data = pd.read_csv('data/grand_master.csv')
# Merge the two data sets
merged_data = pd.concat([challenger_data, grandmaster_data])
merged_data.drop(list(merged_data.filter(regex = 'red')), axis = 1, inplace = True)
pprint.pprint(list(merged_data.keys()))

['gameId',
 'gameDuraton',
 'blueWins',
 'blueFirstBlood',
 'blueFirstTower',
 'blueFirstBaron',
 'blueFirstDragon',
 'blueFirstInhibitor',
 'blueDragonKills',
 'blueBaronKills',
 'blueTowerKills',
 'blueInhibitorKills',
 'blueWardPlaced',
 'blueWardkills',
 'blueKills',
 'blueDeath',
 'blueAssist',
 'blueChampionDamageDealt',
 'blueTotalGold',
 'blueTotalMinionKills',
 'blueTotalLevel',
 'blueAvgLevel',
 'blueJungleMinionKills',
 'blueKillingSpree',
 'blueTotalHeal',
 'blueObjectDamageDealt']


Voyons une à une la pertinence de chacune des données :

* gameId -> id servant à récupérer les données de la partie, inutile pour notre prédiction 
* gameDuration -> durée de la partie, n'influence pas beaucoup le résultat
* blueFirstBlood -> la 1ère équipe effectuant le premier kill de la partie remporte une petite avance d'or.
* blueFirstTower -> la 1ère équipe détruisant la 1ère tour veut dire qu'elle a l'avantage sur ses adversaires et de plus obtient un bonus d'or. Ce critère peut-être décisif dans le cours d'une partie.
* blueFirstBaron -> le premier Baron Nashor apporte à l'équipe un gros avantage de push mais n'est pas essentiel à la victoire
* blueFirstDragon -> donne un bonus de caractéristique à l'équipe le tuant. Le 1er n'est pas si important, le reste des dragons l'est plus.
* blueFirstInhibitor -> une fois 3 tourelles détruites, l'équipe commence à arriver à la base ennemie, si elle détruise l'inhibiteur, les minions deviennent plus puissante et rajoute une pression de plus pour l'équipe ennemie.
* blueDragonKills -> Beaucoup plus déterminant que le premier Dragon. Les bonus s'ajoutant, ils donnent un gros avantage.
* blueBaronKills -> 


In [20]:
correled_wins = merged_data.corr()['blueWins'][:-1]
correled_wins

gameId                    -0.007261
gameDuraton               -0.013576
blueWins                   1.000000
blueFirstBlood             0.203778
blueFirstTower             0.456809
blueFirstBaron             0.363157
blueFirstDragon            0.265396
blueFirstInhibitor         0.646801
blueDragonKills            0.454370
blueBaronKills             0.350556
blueTowerKills             0.711013
blueInhibitorKills         0.563335
blueWardPlaced             0.029292
blueWardkills              0.064710
blueKills                  0.427080
blueDeath                 -0.424377
blueAssist                 0.340589
blueChampionDamageDealt    0.163875
blueTotalGold              0.259223
blueTotalMinionKills       0.070001
blueTotalLevel             0.204681
blueAvgLevel               0.204681
blueJungleMinionKills      0.196658
blueKillingSpree           0.397397
blueTotalHeal              0.203508
Name: blueWins, dtype: float64